# Example Usage: Modular API Demo

This notebook demonstrates how to use the modular classes from the `src/` package.

In [ ]:
import sys
sys.path.insert(0, '../src')

from sentiment_analyzer import SentimentAnalyzer
from technical_analyzer import TAProcessor
from data_loader import DataLoader, DataPreprocessor
from correlation_analyzer import CorrelationAnalyzer

import pandas as pd
import matplotlib.pyplot as plt

## 1. Sentiment Analysis Example

In [ ]:
# Initialize sentiment analyzer
sentiment_analyzer = SentimentAnalyzer(method='ensemble')

# Analyze single text
text = "The stock price surged to new highs with excellent quarterly results!"
score = sentiment_analyzer.analyze(text)
category = sentiment_analyzer.categorize_sentiment(score)

print(f"Text: {text}")
print(f"Sentiment Score: {score:.3f}")
print(f"Category: {category}")

In [ ]:
# Analyze batch of texts
texts = [
    "Great earnings report!",
    "Stock plummets on bad news",
    "Company announces merger"
]

results = sentiment_analyzer.analyze_batch(texts)
results

## 2. Technical Analysis Example

In [ ]:
# Initialize technical analyzer
ta_processor = TAProcessor()

# Load data
loader = DataLoader('../data')
stock_df = loader.load_stock_data('AAPL')

# Process with all indicators
processed_df = ta_processor.process_stock_data(stock_df)

print(f"Added {len(processed_df.columns) - len(stock_df.columns)} technical indicators")
processed_df[['Date', 'Close', 'SMA_20', 'RSI', 'MACD']].tail()

In [ ]:
# Get indicator summary
summary = ta_processor.get_indicator_summary(processed_df)
pd.DataFrame(summary).T

## 3. Data Loading & Preprocessing Example

In [ ]:
# Load news data
news_df = loader.load_news_data()

# Preprocess
preprocessor = DataPreprocessor()
news_df = preprocessor.normalize_dates(news_df, 'date')
news_df = preprocessor.add_time_features(news_df, 'date')

print(f"Loaded {len(news_df)} news articles")
news_df[['headline', 'trading_date', 'year', 'month']].head()

## 4. Correlation Analysis Example

In [ ]:
# Analyze sentiment
news_df = sentiment_analyzer.analyze_dataframe(news_df, 'headline')

# Aggregate by date
daily_sentiment = preprocessor.aggregate_by_date(
    news_df,
    'trading_date',
    {'sentiment_score': ['mean', 'count']}
)
daily_sentiment.columns = ['trading_date', 'sentiment_mean', 'news_count']

# Prepare stock data
stock_df = preprocessor.normalize_dates(stock_df, 'Date')
stock_df['Daily_Return'] = ta_processor.calculate_daily_return(stock_df['Close'])

# Merge
merged_df = preprocessor.merge_sentiment_stock(daily_sentiment, stock_df)

print(f"Merged data: {len(merged_df)} days")
merged_df[['trading_date', 'sentiment_mean', 'Daily_Return']].head()

In [ ]:
# Calculate correlation
corr_analyzer = CorrelationAnalyzer()

result = corr_analyzer.analyze_correlation(
    merged_df,
    'sentiment_mean',
    'Daily_Return',
    method='pearson'
)

print(f"Correlation: {result['correlation']:.4f} {result['significance']}")
print(f"P-value: {result['p_value']:.6f}")
print(f"Strength: {corr_analyzer.get_correlation_strength(result['correlation'])}")

In [ ]:
# Lagged correlation
lagged_results = corr_analyzer.lagged_correlation(
    merged_df,
    'sentiment_mean',
    'Daily_Return',
    max_lag=5
)

lagged_results

In [ ]:
# Visualize
fig = corr_analyzer.plot_correlation_scatter(
    merged_df,
    'sentiment_mean',
    'Daily_Return',
    title='Sentiment vs Stock Returns (AAPL)'
)
plt.show()

## Summary

This notebook demonstrates the modular API design:

1. **SentimentAnalyzer**: Clean API for sentiment analysis
2. **TAProcessor**: Technical indicator calculations
3. **DataLoader/DataPreprocessor**: Data handling utilities
4. **CorrelationAnalyzer**: Statistical analysis tools

All classes are reusable, testable, and follow OOP principles.